<a href="https://colab.research.google.com/github/adimnicky/Capstone-Project/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community
!pip install replicate
!pip install pandas

In [ ]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
import pandas as pd

# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
model=model,
replicate_api_token=api_token,
)

In [ ]:
# Load dataset from CSV
df = pd.read_csv("Tweets.csv")

# Define a function to split the list into batches
def batch_list(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

# Define the batch size (adjust as needed based on the model's context window)
batch_size = 100 # You can adjust this number

# Create batches of reviews
review_batches = list(batch_list(customer_reviews_dataset, batch_size))

# Process each batch and store the responses
responses = []
for i, batch in enumerate(review_batches):
    # Use the prompt_template defined earlier
    reviews_text_batch = "\n".join([f"Review {j+1}: {review}" for j, review in enumerate(batch)])
    prompt_batch = prompt_template.format(reviews_text_batch=reviews_text_batch)

    print(f"Processing batch {i+1}/{len(review_batches)}...")
    response_batch = output.invoke(prompt_batch)
    responses.append(response_batch)

# Combine the responses (this will depend on the desired output format)
# For now, let's just print each response
for i, response in enumerate(responses):
    print(f"\nResponse for batch {i+1}:\n")
    print(response)

In [ ]:
# Define refined prompt
refined_prompt = f"""
Classify these reviews as positive, negative, or mixed, and tag
relevant categories (late flight, or rude service):
{response_batch}
"""
# Invoke the model with the example prompt
response = output.invoke(refined_prompt)
# Print the response
print("Granite Model Refined Response:\n")
print(response)

In [ ]:
# Define the prompt to complete the task in 2 steps
multitask_prompt = f"""
Complete the task in 2 steps.
Step 1: Classify these reviews as positive, negative, or mixed.
Step 2: For each review, identify relevant categories: battery
life, screen quality, or performance.
{response_batch}
"""
response = output.invoke(multitask_prompt)
print("Granite Model Response:\n")
print(response)

In [ ]:
# Define the example to guide the model
formatted_prompt = f"""
Classify these reviews as Positive, Negative, or Mixed, and tag
relevant categories. Use this format:
- Sentiment: [Sentiment]
- Categories: [Categories].
{response_batch}
"""
# Invoke the model with prompt
response = output.invoke(formatted_prompt)
# Print the response
print("Granite Model Formatted Response:\n")
print(response)